<a href="https://colab.research.google.com/github/dilanHewawitharana/Intro-to-Deep-Learning-with-PyTorch/blob/master/02_insurance_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [256]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('11dezh1XoSevjh7ED7q53tUvN5h5SlL87')

# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [257]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
!pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [258]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [259]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [260]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [261]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [262]:
your_name = "dilan"

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [263]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [264]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
12,23,male,36.12000,0,no,1972.990440
306,28,female,28.87500,2,no,21791.884820
318,44,female,29.02725,0,no,8014.890114
815,20,female,33.03300,0,no,2028.163752
157,18,male,26.43375,0,yes,16759.634670


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [265]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [266]:
num_cols = dataframe.shape[1]
print(num_cols)

6


**Q: What are the column titles of the input variables?**

In [267]:
input_cols = ["age", "sex", "bmi", "children", "smoker"]

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [268]:
categorical_cols = ["sex", "smoker"]

**Q: What are the column titles of output/target variable(s)?**

In [269]:
output_cols = ["charges"]

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [270]:
# Write your answer here
charge_col = dataframe["charges"]
print("min_value : {}".format(min(charge_col)))
print("max_value : {}".format(max(charge_col)))
print("avg_value : {}".format(sum(charge_col)/len(charge_col)))

min_value : 1211.623812
max_value : 68872.06225080001
avg_value : 14422.344658803895


Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [271]:
!pip install jovian --upgrade -q

In [272]:
import jovian

In [273]:
jovian.commit(project=project_name)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [274]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy(dtype='float32')
    targets_array = dataframe1[output_cols].to_numpy(dtype='float32')
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [275]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[23.     ,  1.     , 36.12   ,  0.     ,  0.     ],
        [28.     ,  0.     , 28.875  ,  2.     ,  0.     ],
        [44.     ,  0.     , 29.02725,  0.     ,  0.     ],
        ...,
        [35.     ,  1.     , 38.5035 ,  1.     ,  1.     ],
        [59.     ,  0.     , 32.9175 ,  0.     ,  0.     ],
        [62.     ,  0.     , 34.61325,  3.     ,  0.     ]], dtype=float32),
 array([[ 1972.9905],
        [21791.885 ],
        [ 8014.89  ],
        ...,
        [42956.22  ],
        [13631.954 ],
        [16861.168 ]], dtype=float32))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [276]:
inputs = torch.from_numpy(inputs_array)
targets = torch.from_numpy(targets_array)

In [277]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [278]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [279]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [280]:
batch_size = 128

In [281]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [282]:
for xb, yb in train_loader:
    print("inputs:", len(xb))
    print("targets:", len(yb))
    break

inputs: 128
targets: 128


Let's save our work by committing to Jovian.

In [283]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [284]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [285]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size) # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb) # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss_fn = F.l1_loss
        loss = loss_fn(out, targets)                       # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss_fn = F.l1_loss
        loss = loss_fn(out, targets)                          # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [286]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [287]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3018, -0.2009, -0.3760, -0.1328, -0.2950]], requires_grad=True),
 Parameter containing:
 tensor([0.3343], requires_grad=True)]

One final commit before we train the model.

In [288]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [289]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [290]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 15308.49609375}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [291]:
epochs = 1000
lr = 1e-2
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 11247.2441
Epoch [40], val_loss: 9535.1602
Epoch [60], val_loss: 9089.2090
Epoch [80], val_loss: 8988.4180
Epoch [100], val_loss: 8945.7539
Epoch [120], val_loss: 8914.6094
Epoch [140], val_loss: 8884.7578
Epoch [160], val_loss: 8855.5117
Epoch [180], val_loss: 8826.7988
Epoch [200], val_loss: 8798.4639
Epoch [220], val_loss: 8771.1387
Epoch [240], val_loss: 8744.5654
Epoch [260], val_loss: 8717.9385
Epoch [280], val_loss: 8692.6016
Epoch [300], val_loss: 8667.4922
Epoch [320], val_loss: 8642.7852
Epoch [340], val_loss: 8618.4980
Epoch [360], val_loss: 8594.4727
Epoch [380], val_loss: 8570.5957
Epoch [400], val_loss: 8546.9893
Epoch [420], val_loss: 8523.4580
Epoch [440], val_loss: 8501.1064
Epoch [460], val_loss: 8479.1953
Epoch [480], val_loss: 8459.8535
Epoch [500], val_loss: 8442.0654
Epoch [520], val_loss: 8426.0479
Epoch [540], val_loss: 8410.5742
Epoch [560], val_loss: 8395.2979
Epoch [580], val_loss: 8380.8145
Epoch [600], val_loss: 8366.7559
Epoch [620], 

In [292]:
epochs = 1000
lr = 1e-2
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 8268.3008
Epoch [40], val_loss: 8267.3350
Epoch [60], val_loss: 8266.7617
Epoch [80], val_loss: 8267.2852
Epoch [100], val_loss: 8266.4131
Epoch [120], val_loss: 8266.4375
Epoch [140], val_loss: 8265.2207
Epoch [160], val_loss: 8264.6680
Epoch [180], val_loss: 8265.3418
Epoch [200], val_loss: 8265.3955
Epoch [220], val_loss: 8264.4805
Epoch [240], val_loss: 8263.9658
Epoch [260], val_loss: 8264.1738
Epoch [280], val_loss: 8264.0000
Epoch [300], val_loss: 8263.2998
Epoch [320], val_loss: 8263.8750
Epoch [340], val_loss: 8263.2227
Epoch [360], val_loss: 8262.4834
Epoch [380], val_loss: 8262.7305
Epoch [400], val_loss: 8262.7227
Epoch [420], val_loss: 8262.4180
Epoch [440], val_loss: 8261.8926
Epoch [460], val_loss: 8261.5273
Epoch [480], val_loss: 8261.2539
Epoch [500], val_loss: 8261.1602
Epoch [520], val_loss: 8260.6572
Epoch [540], val_loss: 8259.7012
Epoch [560], val_loss: 8259.7080
Epoch [580], val_loss: 8259.4775
Epoch [600], val_loss: 8259.3418
Epoch [620], v

In [293]:
epochs = 1000
lr = 1e-2
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 8253.4238
Epoch [40], val_loss: 8253.6475
Epoch [60], val_loss: 8253.1973
Epoch [80], val_loss: 8252.8633
Epoch [100], val_loss: 8253.0020
Epoch [120], val_loss: 8252.6934
Epoch [140], val_loss: 8252.6758
Epoch [160], val_loss: 8252.2393
Epoch [180], val_loss: 8251.5195
Epoch [200], val_loss: 8251.2295
Epoch [220], val_loss: 8251.4404
Epoch [240], val_loss: 8251.1260
Epoch [260], val_loss: 8250.5293
Epoch [280], val_loss: 8250.5596
Epoch [300], val_loss: 8250.3672
Epoch [320], val_loss: 8249.9668
Epoch [340], val_loss: 8249.9102
Epoch [360], val_loss: 8250.3809
Epoch [380], val_loss: 8249.3008
Epoch [400], val_loss: 8250.1377
Epoch [420], val_loss: 8249.6934
Epoch [440], val_loss: 8249.5273
Epoch [460], val_loss: 8249.1426
Epoch [480], val_loss: 8248.8770
Epoch [500], val_loss: 8248.9834
Epoch [520], val_loss: 8248.6152
Epoch [540], val_loss: 8248.6484
Epoch [560], val_loss: 8248.1133
Epoch [580], val_loss: 8247.9336
Epoch [600], val_loss: 8247.6484
Epoch [620], v

In [294]:
epochs = 1000
lr = 1e-2
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 8243.7900
Epoch [40], val_loss: 8243.6260
Epoch [60], val_loss: 8243.7578
Epoch [80], val_loss: 8243.9795
Epoch [100], val_loss: 8243.6172
Epoch [120], val_loss: 8243.0859
Epoch [140], val_loss: 8242.3994
Epoch [160], val_loss: 8242.1768
Epoch [180], val_loss: 8242.1367
Epoch [200], val_loss: 8242.1797
Epoch [220], val_loss: 8242.0312
Epoch [240], val_loss: 8240.7969
Epoch [260], val_loss: 8242.0283
Epoch [280], val_loss: 8241.7129
Epoch [300], val_loss: 8241.7412
Epoch [320], val_loss: 8241.2061
Epoch [340], val_loss: 8241.0244
Epoch [360], val_loss: 8240.9922
Epoch [380], val_loss: 8240.7363
Epoch [400], val_loss: 8240.3154
Epoch [420], val_loss: 8239.6689
Epoch [440], val_loss: 8239.9385
Epoch [460], val_loss: 8239.4092
Epoch [480], val_loss: 8239.4277
Epoch [500], val_loss: 8239.2705
Epoch [520], val_loss: 8238.7031
Epoch [540], val_loss: 8238.6738
Epoch [560], val_loss: 8238.0938
Epoch [580], val_loss: 8237.7715
Epoch [600], val_loss: 8237.7871
Epoch [620], v

In [295]:
epochs = 1000
lr = 1e-2
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 8234.5107
Epoch [40], val_loss: 8233.9092
Epoch [60], val_loss: 8232.7246
Epoch [80], val_loss: 8232.2139
Epoch [100], val_loss: 8232.0859
Epoch [120], val_loss: 8231.9980
Epoch [140], val_loss: 8231.6934
Epoch [160], val_loss: 8231.1699
Epoch [180], val_loss: 8231.9375
Epoch [200], val_loss: 8231.2578
Epoch [220], val_loss: 8231.6357
Epoch [240], val_loss: 8231.1562
Epoch [260], val_loss: 8230.7754
Epoch [280], val_loss: 8230.2500
Epoch [300], val_loss: 8230.7861
Epoch [320], val_loss: 8229.7012
Epoch [340], val_loss: 8230.0742
Epoch [360], val_loss: 8229.5957
Epoch [380], val_loss: 8229.2432
Epoch [400], val_loss: 8229.8945
Epoch [420], val_loss: 8228.8340
Epoch [440], val_loss: 8229.7090
Epoch [460], val_loss: 8228.5273
Epoch [480], val_loss: 8228.5723
Epoch [500], val_loss: 8228.1387
Epoch [520], val_loss: 8228.6445
Epoch [540], val_loss: 8228.9336
Epoch [560], val_loss: 8227.5254
Epoch [580], val_loss: 8227.9316
Epoch [600], val_loss: 8228.2969
Epoch [620], v

**Q: What is the final validation loss of your model?**

In [296]:
val_loss = 8224.1152

Let's log the final validation loss to Jovian and commit the notebook

In [297]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [298]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [301]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)               # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [302]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([27.0000,  1.0000, 24.2550,  0.0000,  0.0000])
Target: tensor([2682.4348])
Prediction: tensor([5266.7627])


In [303]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([45.0000,  1.0000, 41.7952,  0.0000,  0.0000])
Target: tensor([8044.2764])
Prediction: tensor([8692.8184])


In [304]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([31.0000,  1.0000, 41.4645,  1.0000,  0.0000])
Target: tensor([4185.7930])
Prediction: tensor([4984.5625])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [305]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression
[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'